<a href="https://colab.research.google.com/github/awsdevguru/PearsonMLFoundations/blob/main/2_3_03_Data_Normalization_Scaling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Normalization & Scaling

## 1. Objectives

* Why scaling matters
* How to apply StandardScaler, MinMaxScaler, RobustScaler
* How scaling interacts with outliers
* How to visualize and compare scaling
* How to integrate scaling into Pipelines & ColumnTransformers

## 2. Setup

In [ ]:
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

## 2. Load Titanic Dataset

In [ ]:
df = sns.load_dataset("titanic")
df.head()

## 3. Select Relevant Columns

In [ ]:
cols = ["survived", "age", "fare", "sex", "class"]
df = df[cols].copy()
df.head()

## 4. Clean Missing Values

In [ ]:
df_clean = df.dropna(subset=["age", "fare", "sex", "class"])
df_clean.isna().sum()

## 5. Basic Exploration (Check Scale Differences)

In [ ]:
df_clean[["age","fare"]].describe()


In [ ]:
sns.boxplot(data=df_clean[["age","fare"]])
plt.title("Age vs Fare: Different Scales & Outliers")
plt.show()

## 6. Apply Three Scaling Methods

In [ ]:
scalers = {
    "StandardScaler": StandardScaler(),
    "MinMaxScaler": MinMaxScaler(),
    "RobustScaler": RobustScaler()
}

scaled_results = {}

for name, scaler in scalers.items():
    scaled = scaler.fit_transform(df_clean[["age","fare"]])
    scaled_results[name] = pd.DataFrame(
        scaled,
        columns=["age_scaled", "fare_scaled"]
    )

pd.concat(scaled_results, axis=1).head()

## 7. Visualize the Effects of Scaling

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(18,5))

sns.histplot(scaled_results["StandardScaler"]["fare_scaled"], kde=True, ax=ax[0])
ax[0].set_title("StandardScaler — Fare")

sns.histplot(scaled_results["MinMaxScaler"]["fare_scaled"], kde=True, ax=ax[1])
ax[1].set_title("MinMaxScaler — Fare")

sns.histplot(scaled_results["RobustScaler"]["fare_scaled"], kde=True, ax=ax[2])
ax[2].set_title("RobustScaler — Fare")

plt.show()


## 8. When to Use Each Scaler

* **StandardScaler:** "Best for linear models (Logistic Reg, SVM).
* **MinMaxScaler:** "Best for neural networks and KNN.
* **RobustScaler:** "Best when data has outliers.

## 9. Train/Test Split for Modeling

In [ ]:
X = df_clean[["age","fare","sex"]]
y = df_clean["survived"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


## 10. Build a Pipeline (Scaling + Logistic Regression)

In [ ]:
from sklearn.preprocessing import OneHotEncoder

num_cols = ["age", "fare"]
cat_cols = ["sex"]

preprocess = ColumnTransformer([
    ("scale", StandardScaler(), num_cols),
    ("encode", OneHotEncoder(), cat_cols)
])

In [ ]:
clf = Pipeline([
    ("prep", preprocess),
    ("model", LogisticRegression(max_iter=200))
])

clf.fit(X_train, y_train)
preds = clf.predict(X_test)

accuracy_score(y_test, preds)


## 11. Summary Table of Scaling Methods

In [ ]:
summary = pd.DataFrame({
    "Scaler": ["Standard", "MinMax", "Robust"],
    "Handles Outliers?": ["No", "No", "Yes"],
    "Changes Distribution?": ["No", "No", "No"],
    "Best Use Case": [
        "Linear/SVM models",
        "Neural networks / KNN",
        "Outlier-heavy data"
    ]
})

summary
